<a href="https://colab.research.google.com/github/zixingguan/25/blob/main/notebooks/W04_postgis2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.2/884.2 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [ ]:
url = "https://open.gishub.org/data/duckdb/nyc_data.db.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://open.gishub.org/data/duckdb/nyc_data.db.zip
To: /content/nyc_data.db.zip
100%|██████████| 8.60M/8.60M [00:00<00:00, 118MB/s]


Extracting files...


'/content/nyc_data.db.zip'

In [ ]:
# 连接到duckDBdb
con = duckdb.connect("nyc_data.db")

In [ ]:
con.install_extension("spatial")
con.load_extension("spatial")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ nyc_census_blocks   │
│ nyc_homicides       │
│ nyc_neighborhoods   │
│ nyc_streets         │
│ nyc_subway_stations │
└─────────────────────┘

1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [ ]:
con.sql("""
SELECT *
FROM nyc_neighborhoods
WHERE name = 'Little Italy';
""")

┌───────────┬──────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ BORONAME  │     NAME     │                                                                                                                                        geom                                                                                                                                         │
│  varchar  │   varchar    │                                                                                                                                      geometry                                                                                                                                       │
├───────────┼──────────────┼───────────────────────────────────────────────────

In [ ]:
con.sql("SELECT * from nyc_subway_stations LIMIT 5")

┌──────────┬────────┬──────────────┬─────────────────┬─────────────────┬────────────────────────────────────────┬──────────────────────────────────┬───────────┬─────────┬─────────┬───────────┬─────────┬─────────┬─────────┬─────────────────────────────────────────────┐
│ OBJECTID │   ID   │     NAME     │    ALT_NAME     │    CROSS_ST     │               LONG_NAME                │              LABEL               │  BOROUGH  │ NGHBHD  │ ROUTES  │ TRANSFERS │  COLOR  │ EXPRESS │ CLOSED  │                    geom                     │
│  double  │ double │   varchar    │     varchar     │     varchar     │                varchar                 │             varchar              │  varchar  │ varchar │ varchar │  varchar  │ varchar │ varchar │ varchar │                  geometry                   │
├──────────┼────────┼──────────────┼─────────────────┼─────────────────┼────────────────────────────────────────┼──────────────────────────────────┼───────────┼─────────┼─────────┼───────────┼─

In [ ]:
con.sql("""
SELECT s.NAME AS station_name, s.ROUTES
FROM nyc_subway_stations s
JOIN nyc_neighborhoods n ON ST_Within(s.geom, n.geom)
WHERE n.NAME = 'Little Italy';
""")

┌──────────────┬─────────┐
│ station_name │ ROUTES  │
│   varchar    │ varchar │
├──────────────┼─────────┤
│ Spring St    │ 6       │
└──────────────┴─────────┘

2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [ ]:
con.sql("""
SELECT DISTINCT n.NAME AS neighborhood_name
FROM nyc_subway_stations s
JOIN nyc_neighborhoods n ON ST_Within(s.geom, n.geom)
WHERE s.ROUTES LIKE '%6%';
""")

┌────────────────────┐
│ neighborhood_name  │
│      varchar       │
├────────────────────┤
│ Financial District │
│ Little Italy       │
│ Gramercy           │
│ East Harlem        │
│ Mott Haven         │
│ Chinatown          │
│ Greenwich Village  │
│ Murray Hill        │
│ Midtown            │
│ Upper East Side    │
│ Yorkville          │
│ Hunts Point        │
│ South Bronx        │
│ Soundview          │
│ Parkchester        │
├────────────────────┤
│      15 rows       │
└────────────────────┘

In [ ]:
con.sql("""
SELECT DISTINCT n.name, n.boroname
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE strpos(s.routes,'6') > 0
""")

┌────────────────────┬───────────┐
│        NAME        │ BORONAME  │
│      varchar       │  varchar  │
├────────────────────┼───────────┤
│ Financial District │ Manhattan │
│ Little Italy       │ Manhattan │
│ Upper East Side    │ Manhattan │
│ East Harlem        │ Manhattan │
│ Mott Haven         │ The Bronx │
│ Hunts Point        │ The Bronx │
│ South Bronx        │ The Bronx │
│ Chinatown          │ Manhattan │
│ Greenwich Village  │ Manhattan │
│ Gramercy           │ Manhattan │
│ Murray Hill        │ Manhattan │
│ Midtown            │ Manhattan │
│ Yorkville          │ Manhattan │
│ Soundview          │ The Bronx │
│ Parkchester        │ The Bronx │
├────────────────────┴───────────┤
│ 15 rows              2 columns │
└────────────────────────────────┘

3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [ ]:
con.sql("SELECT * from nyc_census_blocks LIMIT 5")

┌─────────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬───────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│      BLKID      │ POPN_TOTAL │ POPN_WHITE │ POPN_BLACK │ POPN_NATIV │ POPN_ASIAN │ POPN_OTHER │   BORONAME    │               

In [ ]:
con.sql("""
SELECT SUM(c.POPN_TOTAL) AS total_evacuated
FROM nyc_census_blocks c
JOIN nyc_neighborhoods n ON ST_intersects(c.geom, n.geom)
WHERE n.name = 'Battery Park';
""")

┌─────────────────┐
│ total_evacuated │
│     int128      │
├─────────────────┤
│           17153 │
└─────────────────┘

4. **What neighborhood has the highest population density (persons/km2)?**


In [ ]:
con.sql("SELECT * from nyc_neighborhoods LIMIT 5")

┌───────────┬──────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
con.sql("""

SELECT
  n.name,
  Sum(c.popn_total) / (ST_Area(n.geom) / 1000000.0) AS popn_per_sqkm
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_Intersects(c.geom, n.geom)
GROUP BY n.name, n.geom
ORDER BY popn_per_sqkm DESC LIMIT 1;
""")

┌───────────────────┬───────────────────┐
│       NAME        │   popn_per_sqkm   │
│      varchar      │      double       │
├───────────────────┼───────────────────┤
│ North Sutton Area │ 68435.13283772678 │
└───────────────────┴───────────────────┘

When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [1]:
%pip install duckdb duckdb-engine jupysql

In [2]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [3]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [4]:
# 第一步：创建 ais 表
%%sql
CREATE TABLE ais (
    vessel_id INTEGER PRIMARY KEY,
    vessel_name VARCHAR,
    vsl_descr VARCHAR,
    dwt FLOAT,
    v_length FLOAT,
    draft FLOAT,
    sog FLOAT,
    date TIMESTAMP,
    lat FLOAT,
    lon FLOAT,
    geom GEOMETRY
);

,Success


In [5]:
# 读取 CSV 文件
url = 'https://storage.googleapis.com/qm2/casa0025_ships.csv'
df = pd.read_csv(url)

# 删除重复的 vesselid
df_unique = df.drop_duplicates(subset='vesselid')

# 将唯一记录写入新的 CSV 文件
df_unique.to_csv('unique_casa0025_ships.csv', index=False)

In [6]:
%%sql
COPY ais FROM 'unique_casa0025_ships.csv' (HEADER);

,Success


In [7]:
%%sql
ALTER TABLE ais ADD COLUMN geom_wkt TEXT;

,Success


In [8]:
%%sql
UPDATE ais
SET geom_wkt = ST_AsText(geom);

,Success


In [9]:
%%sql
select * from ais limit 5

,vessel_id,vessel_name,vsl_descr,dwt,v_length,draft,sog,date,lat,lon,geom,geom_wkt
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,5.2,2022-07-25 02:53:29,45.151775,36.513329,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (36.5133266666667 45.1517766666667)
1,323648,A Line,bulk carrier,12259.0,109.0,4.7,12.4,2022-06-28 14:31:37,45.082466,36.503517,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (36.5035166666667 45.0824666666667)
2,142540,A Plus-1,general cargo with container capacity,4742.0,130.0,2.7,7.6,2022-08-18 13:14:19,45.086884,36.506268,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (36.5062666666667 45.0868833333333)
3,319402,Abramtsevo,general cargo,2070.0,104.0,3.2,0.0,2022-08-14 08:26:52,45.155277,36.508270,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (36.5082716666667 45.1552766666667)
4,213151,Absheron,general cargo with container capacity,3344.0,116.0,2.8,7.4,2022-06-14 22:24:10,45.078552,36.507748,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",POINT (36.5077466666667 45.0785533333333)


In [10]:
#第三步：创建 vinfo 表
%%sql
CREATE TABLE vinfo (
    vesselid VARCHAR PRIMARY KEY,
    vessel_name VARCHAR,
    vsl_descr VARCHAR,
    dwt NUMERIC,
    v_length NUMERIC,
    draft NUMERIC,
    geom TEXT
);

,Success


In [11]:
%%sql
select * from vinfo limit 5

,vesselid,vessel_name,vsl_descr,dwt,v_length,draft,geom


In [12]:
%%sql
INSERT INTO vinfo (vesselid, vessel_name, vsl_descr, dwt, v_length, draft, geom)
SELECT vessel_id, vessel_name, vsl_descr, dwt, v_length, draft, geom_wkt
FROM ais;

,Success


In [13]:
%%sql
select * from vinfo limit 5

,vesselid,vessel_name,vsl_descr,dwt,v_length,draft,geom
0,350053,30 Let Pobedy,general cargo,5150.0,NaN,3.5,POINT (36.5133266666667 45.1517766666667)
1,323648,A Line,bulk carrier,12259.0,109.0,4.7,POINT (36.5035166666667 45.0824666666667)
2,142540,A Plus-1,general cargo with container capacity,4742.0,130.0,2.7,POINT (36.5062666666667 45.0868833333333)
3,319402,Abramtsevo,general cargo,2070.0,104.0,3.2,POINT (36.5082716666667 45.1552766666667)
4,213151,Absheron,general cargo with container capacity,3344.0,116.0,2.8,POINT (36.5077466666667 45.0785533333333)


In [14]:
%%sql
ALTER TABLE ais ADD COLUMN geom_geometry GEOMETRY;
UPDATE ais SET geom_geometry = ST_GeomFromText(geom_wkt);

,Success


In [15]:
%%sql
ALTER TABLE vinfo ADD COLUMN geom_geometry GEOMETRY;
UPDATE vinfo SET geom_geometry = ST_GeomFromText(geom);

,Success


## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.

In [19]:
%%sql
WITH filtered_ais AS (
    SELECT
        a1.vessel_id AS ship1_id,
        a2.vessel_id AS ship2_id,
        a1.date AS ship1_time,
        a2.date AS ship2_time,
        ST_Distance(a1.geom, a2.geom) AS distance,
        a1.sog AS ship1_speed,
        a2.sog AS ship2_speed
    FROM
        ais a1
    JOIN
        ais a2 ON a1.vessel_id != a2.vessel_id  -- 确保不与自己连接
    WHERE
        ST_Distance(a1.geom, a2.geom) < 500  -- 距离小于500米
        AND (EXTRACT(EPOCH FROM a2.date) - EXTRACT(EPOCH FROM a1.date)) / 3600 >= 2  -- 时间差大于2小时
        AND a1.sog < 1  -- 船1速度小于1节
        AND a2.sog < 1  -- 船2速度小于1节
)

SELECT * FROM filtered_ais;

,ship1_id,ship2_id,ship1_time,ship2_time,distance,ship1_speed,ship2_speed
0,330665,319402,2022-06-01 00:16:26,2022-08-14 08:26:52,0.073310,0.0,0.0
1,265327,319402,2022-06-20 07:07:55,2022-08-14 08:26:52,0.006772,0.0,0.0
2,352210,319402,2022-06-01 00:26:44,2022-08-14 08:26:52,0.040483,0.1,0.0
3,361341,319402,2022-06-01 00:24:53,2022-08-14 08:26:52,0.033562,0.0,0.0
4,11007170,319402,2022-06-22 10:10:11,2022-08-14 08:26:52,0.023540,0.8,0.0
...,...,...,...,...,...,...,...
4811,322462,312965,2022-06-22 11:17:53,2022-07-16 04:59:02,0.052125,0.8,0.2
4812,215316,312965,2022-06-20 17:11:57,2022-07-16 04:59:02,0.104444,0.4,0.2
4813,171630,312965,2022-06-27 17:27:36,2022-07-16 04:59:02,0.022621,0.0,0.2
4814,215477,312965,2022-06-01 00:26:19,2022-07-16 04:59:02,0.111309,0.0,0.2
